In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [43]:
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [44]:
# hyperparameters
batch_size = 32
epochs = 2
lr = 0.001

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [45]:
train_df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv', dtype = np.float32)
test_df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv', dtype = np.float32)
sample_sub = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')

train_df.shape

(42000, 785)

In [46]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
y_train = train_df.label.values

X_train = train_df.drop('label', axis = 1).values / 255

print(X_train.shape, y_train.shape)

(42000, 784) (42000,)


In [48]:
y_train = torch.from_numpy(y_train).type(torch.LongTensor)

In [49]:
X_train = X_train.reshape((-1, 1, 28, 28))
X_train.shape

(42000, 1, 28, 28)

In [50]:
X_train = np.stack((X_train,) * 3, axis = 1)
X_train.shape

(42000, 3, 1, 28, 28)

In [51]:
X_train = np.squeeze(X_train, axis = 2)
X_train.shape

(42000, 3, 28, 28)

In [52]:
Transform = transforms.Compose([transforms.ToTensor()])

In [53]:
class GetData(Dataset):
    def __init__(self, X_train, y_train, Transform):
        self.X = X_train
        self.transform = Transform
        self.y = y_train
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.transform(self.X[index]).permute(1, 2, 0).contiguous(), self.y[index]

In [54]:
trainset = GetData(X_train, y_train, Transform)
trainloader = DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 4)

In [55]:
trainset[21][0].shape

torch.Size([3, 28, 28])

In [56]:
next(iter(trainloader))[0].shape

torch.Size([32, 3, 28, 28])

In [57]:
model = torchvision.models.resnet152()
print(model)
model.fc = nn.Linear(2048, 10, bias = True)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [58]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [59]:
from tqdm import tqdm

In [60]:
for epoch in range(3):
    tr_loss = 0.0
    
    model = model.train()
    
    for i, (images, labels) in enumerate(trainloader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        logits = model(images)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i % 100 == 0):
            print(f'Step = {i} Loss = {loss.item()}')
        
        tr_loss += loss.detach().item()
        
    model.eval()
    print(f'Epoch: {epoch} , Loss: {tr_loss / i}')

Step = 0 Loss = 2.733882427215576
Step = 100 Loss = 0.7357672452926636
Step = 200 Loss = 0.31662869453430176
Step = 300 Loss = 0.3075426518917084
Step = 400 Loss = 0.35266023874282837
Step = 500 Loss = 0.3931894600391388
Step = 600 Loss = 0.5163302421569824
Step = 700 Loss = 0.19441714882850647
Step = 800 Loss = 0.18808287382125854
Step = 900 Loss = 0.04685844108462334
Step = 1000 Loss = 0.13408297300338745
Step = 1100 Loss = 0.18958823382854462
Step = 1200 Loss = 0.03083573468029499
Step = 1300 Loss = 0.17118535935878754
Epoch: 0 , Loss: 0.4751039957613457
Step = 0 Loss = 0.3025141656398773
Step = 100 Loss = 0.05400572344660759
Step = 200 Loss = 0.09793581813573837
Step = 300 Loss = 0.061430588364601135
Step = 400 Loss = 0.7014527320861816
Step = 500 Loss = 0.12346666306257248
Step = 600 Loss = 0.1150440126657486
Step = 700 Loss = 0.029202140867710114
Step = 800 Loss = 0.10740385949611664
Step = 900 Loss = 0.023970497772097588
Step = 1000 Loss = 0.16026949882507324
Step = 1100 Loss = 

In [61]:
X_test = (test_df.values / 255).reshape((-1, 1, 28, 28))
X_test.shape

(28000, 1, 28, 28)

In [62]:
X_test = np.stack((X_test, ) * 3, axis = 1)
X_test = np.squeeze(X_test, axis = 2)
X_test.shape

(28000, 3, 28, 28)

In [65]:
with torch.no_grad():
    model.eval()
    out = model(torch.from_numpy(X_test).to(device)).cpu().argmax(dim = 1)
    sample_sub['Label'] = out

In [66]:
sample_sub.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [67]:
sample_sub.to_csv('nachiket_submission.csv', index = False)